In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DATA_DIR = "/kaggle/input/yelp-review-dataset/yelp_review_polarity_csv/"

data = pd.read_csv(DATA_DIR + "train.csv",header=None)

In [ ]:
data.head()

In [ ]:
data.columns = ["rating","text"]

In [ ]:
data.head()

In [ ]:
data["rating"].value_counts()

In [ ]:
data.text[0], data.rating[0]

In [ ]:
data.text[1], data.rating[1]

**TFIDF + Logistic Regression**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = data.rating.values

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(data.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
print (xtrain.shape)
print (xvalid.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
%%time
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

tfv.fit(list(xtrain) + list(xvalid))
xtrain_transform  = tfv.transform(xtrain)
xvalid_transform  = tfv.transform(xvalid)


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C =1.0 )
clf.fit(xtrain_transform, ytrain)
predictions = clf.predict_proba(xvalid_transform)


In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(yvalid, predictions[:, 1])